In [74]:
import pandas as pd

In [75]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import requests
import time
from collections import defaultdict
from selenium.webdriver.common.by import By

In [76]:
##Options for chromedriver, when using selenium, does not matter when using Collab since it acts more like a remote machine
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [77]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [95]:
cricbuzz_highlights_url='https://www.cricbuzz.com/cricket-full-commentary/45901/gt-vs-lsg-4th-match-indian-premier-league-2022'

In [96]:
driver.get(cricbuzz_highlights_url)

In [97]:
cricbuzz_soup= BeautifulSoup(driver.page_source, 'html.parser')

In [98]:
div_tag=cricbuzz_soup.find_all('div',{'class':"cb-col cb-col-100 ng-scope"})

In [99]:
#class="cb-col cb-col-100 ng-scope"
#class="cb-col cb-col-100 ng-scope"
#class="cb-col cb-col-100 ng-scope"
#class="cb-com-ln ng-binding ng-scope cb-col cb-col-90"
#class="cb-col cb-col-20" match box
#class=cb-nav-subhdr cb-font-12  header 
#class="cb-billing-plans-text cb-team-lft-item" team names

In [116]:
match_metadata_values=[c.next_element.next_element.next_element.text.replace('\xa0','') for c in cricbuzz_soup.find_all('span',{'class':'text-bold'})]

match_metadata_values



['Indian Premier League 2022', 'Wankhede Stadium,Mumbai', 'Mar 28,']

In [121]:
year=match_metadata_values[0].split()[-1]
series="IPL"
venue=match_metadata_values[1]
date=match_metadata_values[2]+year

In [100]:
def match_no():
    team_names=cricbuzz_soup.find('div',{'class':"cb-billing-plans-text cb-team-lft-item"})
    teams=team_names.text.split(',')[1]
    return teams.strip()[0:3]
match_no()

'4th'

In [101]:
def team_names():
    team_names=cricbuzz_soup.find('div',{'class':"cb-billing-plans-text cb-team-lft-item"})
    teams=team_names.text.split(',')[0]
    teams=teams.split('vs')
    teams=[team.strip() for team in teams]
    return teams


In [102]:
def match_info():
    match_details=cricbuzz_soup.find('div',{'class':"cb-col cb-col-20"})
    match_info=""
    for i in match_details:
        match_info+=i.get_text()
    return match_info    


In [103]:
def find_team_subs(team):
    preview_obj=cricbuzz_soup.find_all('p',{'class':'cb-com-ln ng-binding ng-scope cb-col cb-col-90'})
    for tag in preview_obj:
        if tag.b==None:
            pass
        else:
            if "subs" in tag.get_text().lower():
                print(tag.get_text())
                return tag.get_text().split(',')[0:]
find_team_subs(team_b)

In [104]:
def find_team_playing11(team):
    preview_obj=cricbuzz_soup.find_all('p',{'class':'cb-com-ln ng-binding ng-scope cb-col cb-col-90'})
    for tag in preview_obj:
        if tag.b==None:
            pass
        else:
            if team+" (Playing XI)" in tag.get_text():
                return tag.get_text().split(',')[0:]
    

In [105]:
df=pd.DataFrame(columns=['match_no','series','season','team_a','team_b','team_a_11','team_b_11','team_a_subs','team_b_subs','venue','date'])

In [106]:
match =match_no()

df.at[1,'match_no']=match

In [107]:
team_a,team_b=team_names()

In [108]:
df.at[1,'team_a']=team_a
df.at[1,'team_b']=team_b

In [122]:
df.at[1,'team_a_11']=find_team_playing11(team_a)
df.at[1,'team_b_11']=find_team_playing11(team_b)
df.at[1,'team_a_subs']=find_team_subs(team_a)
df.at[1,'team_b_subs']=find_team_subs(team_b)
df.at[1,'series']=series
df.at[1,'season']=year
df.at[1,'venue']=venue
df.at[1,'date']=date

In [123]:
display(df)

,match_no,team_a,team_b,team_a_11,team_b_11,team_a_subs,team_b_subs,series,season,venue,date
1,4th,Gujarat Titans,Lucknow Super Giants,"[Gujarat Titans (Playing XI): Shubman Gill, M...",[Lucknow Super Giants (Playing XI): KL Rahul(c...,None,None,IPL,2022,"Wankhede Stadium,Mumbai","Mar 28,2022"


In [124]:
df.loc[1,'team_a_11']

['Gujarat Titans (Playing XI): Shubman Gill',
 ' Matthew Wade(w)',
 ' Vijay Shankar',
 ' Abhinav Manohar',
 ' Hardik Pandya(c)',
 ' David Miller',
 ' Rahul Tewatia',
 ' Rashid Khan',
 ' Lockie Ferguson',
 ' Varun Aaron',
 ' Mohammed Shami']